In [2]:
import torch
import numpy as np

class OxfordDataset(torch.utils.data.Dataset):
    def __init__(self,
                 truthfile='./data/trolley/data1/syn/vi1.csv',
                 datafile='./data/trolley/data1/syn/imu1.csv',
                 window_samples=200,
                 overlap_samples=0):
        self.sensor = np.loadtxt(datafile,delimiter=',').astype(np.float32)
        self.truth = np.loadtxt(truthfile,delimiter=',').astype(np.float32)
        self.len = self.sensor.shape[0] // (window_samples-overlap_samples)
        self.window_samples = window_samples
        self.overlap_samples = overlap_samples
    def __getitem__(self, index):
        ws = self.window_samples
        pos = index * (ws-self.overlap_samples)
        
        # Sensor Data
        time = self.sensor[pos:(pos+ws),0]
        attitude_rad = self.sensor[pos:(pos+ws),1:4]
        rotation_rate_rad_per_sec = self.sensor[pos:(pos+ws),4:7]
        gravity = self.sensor[pos:(pos+ws),7:10]
        acc = self.sensor[pos:(pos+ws),10:13]
        mag_field_microteslas = self.sensor[pos:(pos+ws),13:16]
        
        # True Data
        time_truth = self.truth[pos:(pos+ws),0] 
        translation = self.truth[pos:(pos+ws),2:5]
        rotation = self.truth[pos:(pos+ws),5:10]
        
        return time, attitude_rad, rotation_rate_rad_per_sec, gravity, acc, mag_field_microteslas, time_truth, translation, rotation

    def __len__(self):
        return self.len

Simple Example that uses attitude and acceleration to estimate delta translation


In [ ]:

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader

is_cuda = torch.cuda.is_available()
if is_cuda:
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

z = OxfordDataset()
loader = DataLoader(z,batch_size=16,pin_memory=True)


class Model(nn.Module):
    def __init__(self, input_size, output_size, hidden_dim, n_layers):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.n_layers = n_layers
        self.rnn = nn.RNN(input_size, hidden_dim, n_layers, batch_first=True)   
        self.fc = nn.Linear(hidden_dim, output_size)
    
    def forward(self, x):
        batch_size = x.size(0)
        hidden = self.init_hidden(batch_size)
        out, hidden = self.rnn(x, hidden)
        out = out.contiguous().view(-1, self.hidden_dim)
        out = self.fc(out)

        return out, hidden
    
    def init_hidden(self, batch_size):
        hidden = torch.zeros(self.n_layers, batch_size, self.hidden_dim).to(device)
        return hidden

model = Model(6,3,128,3)
model.to(device)
n_epochs = 100
lr=0.01
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(1, n_epochs + 1):
    optimizer.zero_grad() 
    
    for batch_ndx, sample in enumerate(loader):
        data_in = torch.cat((sample[1],sample[4]),2).to(device) # concatenate inputs att and acc
        data_in.permute(1,0,2)
        trans = sample[7].to(device)
        delta_trans =torch.cat((torch.zeros(trans.shape[0],1,3,dtype=torch.float32).to(device),(trans[:,1:,:] - trans[:,:-1,:])),1) # get the diff 
        output, hidden = model(data_in)
        loss = criterion(output, delta_trans.view(-1,3))
        loss.backward() # Does backpropagation and calculates gradients
        optimizer.step() # Updates the weights accordingly
    
    if epoch%10 == 0:
        print('Epoch: {}/{}.............'.format(epoch, n_epochs), end=' ')
        print("Loss: {:.4f}".format(loss.item()))


    

Epoch: 10/100............. Loss: 10.6805
Epoch: 20/100............. Loss: 0.1589
Epoch: 30/100............. Loss: 1.5785
Epoch: 40/100............. Loss: 0.7210
Epoch: 50/100............. Loss: 20.4062
Epoch: 60/100............. Loss: 7.9765
Epoch: 70/100............. Loss: 27.4229
Epoch: 80/100............. Loss: 5.4286
